In [105]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
import json
import prettytable
import geopandas as gpd
import fiona

In [2]:
blsSeries = pd.read_excel('../msaBlsSeries.xlsx', sheet_name="blsSeries")
msa = gpd.read_file('../../../data/spatial/msa/usMsaCentroids.geojson')

In [3]:
months = {'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7, 'August':8,
          'September':9,'October':10,'November':11,'December':12}

In [4]:
uRateList = blsSeries['unemploymentRateSeries'].tolist()
lfList = blsSeries['laborForceSeries'].tolist()

In [5]:
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/?registrationkey=6aff0b6bece9458d9ecfb4cd10a3a375'

frames = []
items = [1,2,3,4,5,6,7,8]
start = 0
end = 50

for item in items:
    seriesId = []
    year = []
    month = []
    value = []

    print(item, start, end)
    chunk = uRateList[start:end]
    areas = ','.join(chunk)
    response = requests.post(url, data = {"seriesid":areas,"startyear":"2018","endyear":"2020"})
    
    json_data = json.loads(response.text)
    
    for series in json_data['Results']['series']:
        sid = series['seriesID']
        for item in series['data']:
            seriesId.append( sid )
            year.append( item['year'] )
            month.append ( item['periodName'] )
            value.append( item['value'] )
        
    df = pd.DataFrame({'seriesId':seriesId,'month':month,'year':year,'value':value})
    
    frames.append(df)
    
    start += 50
    end += 50

1 0 50
2 50 100
3 100 150
4 150 200
5 200 250
6 250 300
7 300 350
8 350 400


In [6]:
uRateData = pd.concat( frames )

In [7]:
uRateData['seriesId'].nunique()

381

In [8]:
uRate = blsSeries[['MSA','unemploymentRateSeries']]
uRateData = pd.merge(uRateData,uRate,how='left',left_on='seriesId',right_on='unemploymentRateSeries')
uRateData['Day'] = 1
uRateData['Month'] = uRateData.month.map(months)
uRateData['date'] = pd.to_datetime(uRateData[['Month','Day','year']])
uRateData.sort_values(['seriesId','date'], inplace=True)
uRateData['Month'] = uRateData['month'].str[:3]
uRateData['value'] = uRateData['value'].astype(float)

# Repeat that process but with the total labor force

In [9]:
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/?registrationkey=6aff0b6bece9458d9ecfb4cd10a3a375'

frames = []
items = [1,2,3,4,5,6,7,8]
start = 0
end = 50

for item in items:
    seriesId = []
    year = []
    month = []
    value = []

    print(item, start, end)
    chunk = lfList[start:end]
    areas = ','.join(chunk)
    response = requests.post(url, data = {"seriesid":areas,"startyear":"2018","endyear":"2020"})
    
    json_data = json.loads(response.text)
    
    for series in json_data['Results']['series']:
        sid = series['seriesID']
        for item in series['data']:
            seriesId.append( sid )
            year.append( item['year'] )
            month.append ( item['periodName'] )
            value.append( item['value'] )
        
    df = pd.DataFrame({'seriesId':seriesId,'month':month,'year':year,'value':value})
    
    frames.append(df)
    
    start += 50
    end += 50

1 0 50
2 50 100
3 100 150
4 150 200
5 200 250
6 250 300
7 300 350
8 350 400


In [10]:
lfData = pd.concat( frames )
lfData['seriesId'].nunique()

381

In [11]:
lfSeries = blsSeries[['MSA','laborForceSeries']]
lfData = pd.merge(lfData,lfSeries,how='left',left_on='seriesId',right_on='laborForceSeries')
lfData['Day'] = 1
lfData['Month'] = lfData.month.map(months)
lfData['date'] = pd.to_datetime(lfData[['Month','Day','year']])
lfData.sort_values(['seriesId','date'], inplace=True)
lfData['Month'] = lfData['month'].str[:3]
lfData['value'] = lfData['value'].astype(float)

In [12]:
uRateCurr = uRateData.loc[(uRateData['month'] == 'December') & (uRateData['year'] == '2020')]
uRateLast = uRateData.loc[(uRateData['month'] == 'December') & (uRateData['year'] == '2019')]

uRateCurr = uRateCurr[['MSA','seriesId','value']]
uRateCurr.rename(columns={'value':'December_2020_URate'}, inplace=True)

uRateLast = uRateLast[['seriesId','value']]
uRateLast.rename(columns={'value':'December_2019_URate'}, inplace=True)

uRateSum = pd.merge(uRateCurr, uRateLast, how='left', on='seriesId')
uRateSum['geoid'] = uRateSum['seriesId'].str[5:12]

In [13]:
lfCurr = lfData.loc[(lfData['month'] == 'December') & (lfData['year'] == '2020')]
lfLast = lfData.loc[(lfData['month'] == 'December') & (lfData['year'] == '2019')]

lfCurr = lfCurr[['MSA','seriesId','value']]
lfCurr.rename(columns={'value':'December_2020_LaborForce'}, inplace=True)

lfLast = lfLast[['seriesId','value']]
lfLast.rename(columns={'value':'December_2019_LaborForce'}, inplace=True)

lfSum = pd.merge(lfCurr, lfLast, how='left', on='seriesId')
lfSum['geoid'] = lfSum['seriesId'].str[5:12]

In [14]:
msa.head(2)

,geoid,cbsa,area,geometry
0,1312020,12020,"Athens-Clarke County, GA",POINT (-83.21379 33.94901)
1,1312060,12060,"Atlanta-Sandy Springs-Alpharetta, GA",POINT (-84.39957 33.69277)


In [15]:
lfSum['cbsa'] = lfSum['geoid'].str[-5:]
uRateSum['cbsa'] = uRateSum['geoid'].str[-5:]

In [16]:
lfSum.head(2)

,MSA,seriesId,December_2020_LaborForce,December_2019_LaborForce,geoid,cbsa
0,"Anniston-Oxford, AL",LAUMT011150000000006,44289.0,46320.0,0111500,11500
1,"Auburn-Opelika, AL",LAUMT011222000000006,75408.0,77615.0,0112220,12220


In [17]:
uRateSum.head(2)

,MSA,seriesId,December_2020_URate,December_2019_URate,geoid,cbsa
0,"Anniston-Oxford, AL",LAUMT011150000000003,4.4,3.0,0111500,11500
1,"Auburn-Opelika, AL",LAUMT011222000000003,3.0,2.2,0112220,12220


In [18]:
lfSum.shape

(381, 6)

In [19]:
uRateSum.shape

(381, 6)

In [20]:
sumStats = pd.merge(lfSum,uRateSum, how='left', on='cbsa')

In [21]:
sumStats.head()

,MSA_x,seriesId_x,December_2020_LaborForce,December_2019_LaborForce,geoid_x,cbsa,MSA_y,seriesId_y,December_2020_URate,December_2019_URate,geoid_y
0,"Anniston-Oxford, AL",LAUMT011150000000006,44289.0,46320.0,0111500,11500,"Anniston-Oxford, AL",LAUMT011150000000003,4.4,3.0,0111500
1,"Auburn-Opelika, AL",LAUMT011222000000006,75408.0,77615.0,0112220,12220,"Auburn-Opelika, AL",LAUMT011222000000003,3.0,2.2,0112220
2,"Birmingham-Hoover, AL",LAUMT011382000000006,543586.0,559808.0,0113820,13820,"Birmingham-Hoover, AL",LAUMT011382000000003,3.5,2.2,0113820
3,"Daphne-Fairhope-Foley, AL",LAUMT011930000000006,91638.0,95993.0,0119300,19300,"Daphne-Fairhope-Foley, AL",LAUMT011930000000003,3.4,2.4,0119300
4,"Decatur, AL",LAUMT011946000000006,70864.0,72844.0,0119460,19460,"Decatur, AL",LAUMT011946000000003,2.9,2.3,0119460


In [22]:
sumStats = sumStats[['cbsa','geoid_x','MSA_x','December_2020_LaborForce','December_2020_URate']]

In [23]:
sumStats.head(2)

,cbsa,geoid_x,MSA_x,December_2020_LaborForce,December_2020_URate
0,11500,0111500,"Anniston-Oxford, AL",44289.0,4.4
1,12220,0112220,"Auburn-Opelika, AL",75408.0,3.0


In [24]:
msaSum = pd.merge(sumStats,msa,how='left',on='cbsa')

In [25]:
msaSum['state'] = msaSum['MSA_x'].str[-2:]
msaSum = msaSum[['cbsa','geoid','MSA_x','state','December_2020_LaborForce','December_2020_URate','geometry']]
msaSum.rename(columns={'MSA_x':'msa'}, inplace=True)


In [26]:
crs = {'init': 'epsg:4326'}
msaSum = gpd.GeoDataFrame(msaSum, crs=crs, geometry=msaSum['geometry'])

/Users/mmainza/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [27]:
msaSum.head()

,cbsa,geoid,msa,state,December_2020_LaborForce,December_2020_URate,geometry
0,11500,0111500,"Anniston-Oxford, AL",AL,44289.0,4.4,POINT (-85.82603 33.77143)
1,12220,0112220,"Auburn-Opelika, AL",AL,75408.0,3.0,POINT (-85.35556 32.60114)
2,13820,0113820,"Birmingham-Hoover, AL",AL,543586.0,3.5,POINT (-86.72819 33.40368)
3,19300,0119300,"Daphne-Fairhope-Foley, AL",AL,91638.0,3.4,POINT (-87.74984 30.66097)
4,19460,0119460,"Decatur, AL",AL,70864.0,2.9,POINT (-87.10264 34.49064)


In [28]:
nulls = msaSum[msaSum['December_2020_LaborForce'].isnull()]

In [29]:
nulls.head()

,cbsa,geoid,msa,state,December_2020_LaborForce,December_2020_URate,geometry


In [53]:
msaSum.to_file('../../../data/spatial/msa/usMsaData.geojson', driver="GeoJSON")

In [ ]:
# summarize to get latest and previous year

In [54]:
# write files

msaList = pd.read_excel('../msaBlsSeries.xlsx', sheet_name="msaList")
series = pd.read_excel('../msaBlsSeries.xlsx', sheet_name="blsSeries")

In [55]:
msaList= pd.merge(msaList, series, how='left', on='MSA')

In [56]:
msaList.to_excel('deteleMsaList.xlsx', index=False)

In [57]:
pwd

'/Users/mmainza/Desktop/projects/city-comparison/data/processing'

In [58]:
msa = gpd.read_file('../../../data/spatial/msa/usMsaData.geojson')

In [59]:
data= pd.read_excel('../data.xlsx', sheet_name="msaData")

In [60]:
df = pd.merge(msa,data,how='left',left_on='msa',right_on='MSA')

In [61]:
df.sort_values('msa',inplace=True)

In [63]:
df = df[['cbsa','geoid','msa','STATE','December_2020_LaborForce','December_2020_URate',
         'Avg. Local Sales Tax','State Sales Tax','Total Sales Tax','Property Tax per Capita',
         'Property Tax Rank','Property Tax Throw Back','Taxable Wages Rate','New Employers Rate',
         'Taxable Wage Base','Payroll Tax Notes','Individual Tax Rate','Individual Tax Brackets',
         'Graduated Individual Tax','Franchise Tax Rate','Corporate Tax Rate','Corporate Tax Brackets',
         'Graduated Corporate Tax','CODB','Fortune 500','Fortune 1000','AIRPORT','CODE','NUMBER_AIRPORTS',
         'NS_DOMESTIC','NS_INTERNATIONAL','NS_TOTAL','geometry']]

In [64]:
df['AIRPORT'].fillna('None', inplace=True)
df['CODE'].fillna('None', inplace=True)

/Users/mmainza/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [65]:
crs = {'init': 'epsg:4326'}
msaSum = gpd.GeoDataFrame(df, crs=crs, geometry=df['geometry'])

/Users/mmainza/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [66]:
msaGeo = gpd.read_file('../../../data/spatial/msa/usMsaAlbersMasterNODELETE.json')

In [67]:
msaGeo = msaGeo[['msa','geometry']]

In [68]:
msaSum = pd.merge(msaSum,msaGeo,how='left',on='msa')

In [73]:
del msaSum['geometry_x']
msaSum.rename(columns={'geometry_y':'geometry'}, inplace=True)

In [74]:
crs = {'init': 'epsg:4326'}
msaSum = gpd.GeoDataFrame(msaSum, crs=crs, geometry=msaSum['geometry'])

/Users/mmainza/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [76]:
msaSum.to_file('../../../data/spatial/msa/usMsaData.geojson', driver="GeoJSON")

In [96]:
df = gpd.read_file('../../../data/spatial/msa/usMsaData.geojson')
data = pd.read_excel('../data.xlsx', sheet_name="msaData")
airports = pd.read_excel('../data.xlsx', sheet_name="airService",usecols="A,E")

In [93]:
airports.head()

,msa,NUMBER_AIRPORTS
0,"Birmingham-Hoover, AL",1
1,"Kapaa, HI",1
2,"Bozeman, MT",1
3,"Knoxville, TN",1
4,"Phoenix-Mesa-Scottsdale, AZ",1


In [64]:
df = df[['cbsa','geoid','msa','STATE','December_2020_LaborForce','December_2020_URate','AIRPORT','CODE',
         'NS_DOMESTIC','NS_INTERNATIONAL','NS_TOTAL','geometry']]

In [94]:
file = pd.merge(df,data,how='left',on=['msa','STATE'])

In [107]:
df['NUMBER_AIRPORTS'].fillna(0, inplace=True)
df['NUMBER_AIRPORTS'] = df['NUMBER_AIRPORTS'].astype(int)

In [100]:
crs = {'init': 'epsg:4326'}
df = gpd.GeoDataFrame(df, crs=crs, geometry=df['geometry'])

/Users/mmainza/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [108]:
df.dtypes

cbsa                              object
geoid                             object
msa                               object
STATE                             object
December_2020_LaborForce         float64
December_2020_URate              float64
NUMBER_AIRPORTS                    int64
AIRPORT                           object
CODE                              object
NS_DOMESTIC                        int64
NS_INTERNATIONAL                   int64
NS_TOTAL                           int64
Avg. Local Sales Tax             float64
State Sales Tax                  float64
Total Sales Tax                  float64
Property Tax per Capita          float64
Property Tax Rank                float64
Property Tax Throw Back           object
Taxable Wages Rate                object
New Employers Rate               float64
Taxable Wage Base                float64
Payroll Tax Notes                 object
Individual Tax Rate              float64
Individual Tax Brackets          float64
Graduated Indivi

In [110]:
df.to_file('../../../data/spatial/msa/usMsaData.geojson', driver="GeoJSON")

In [111]:
df.to_csv('deleteMe.csv', index=False)